*Note: You are currently reading this using Google Colaboratory which is a cloud-hosted version of Jupyter Notebook. This is a document containing both text cells for documentation and runnable code cells. If you are unfamiliar with Jupyter Notebook, watch this 3-minute introduction before starting this challenge: https://www.youtube.com/watch?v=inN8seMm7UI*

---

In this challenge, you will create a book recommendation algorithm using **K-Nearest Neighbors**.

You will use the [Book-Crossings dataset](http://www2.informatik.uni-freiburg.de/~cziegler/BX/). This dataset contains 1.1 million ratings (scale of 1-10) of 270,000 books by 90,000 users. 

After importing and cleaning the data, use `NearestNeighbors` from `sklearn.neighbors` to develop a model that shows books that are similar to a given book. The Nearest Neighbors algorithm measures distance to determine the “closeness” of instances.

Create a function named `get_recommends` that takes a book title (from the dataset) as an argument and returns a list of 5 similar books with their distances from the book argument.

This code:

`get_recommends("The Queen of the Damned (Vampire Chronicles (Paperback))")`

should return:

```
[
  'The Queen of the Damned (Vampire Chronicles (Paperback))',
  [
    ['Catch 22', 0.793983519077301], 
    ['The Witching Hour (Lives of the Mayfair Witches)', 0.7448656558990479], 
    ['Interview with the Vampire', 0.7345068454742432],
    ['The Tale of the Body Thief (Vampire Chronicles (Paperback))', 0.5376338362693787],
    ['The Vampire Lestat (Vampire Chronicles, Book II)', 0.5178412199020386]
  ]
]
```

Notice that the data returned from `get_recommends()` is a list. The first element in the list is the book title passed in to the function. The second element in the list is a list of five more lists. Each of the five lists contains a recommended book and the distance from the recommended book to the book passed in to the function.

If you graph the dataset (optional), you will notice that most books are not rated frequently. To ensure statistical significance, remove from the dataset users with less than 200 ratings and books with less than 100 ratings.

The first three cells import libraries you may need and the data to use. The final cell is for testing. Write all your code in between those cells.

In [ ]:
# import libraries (you may add additional imports but you may not have to)
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors
import matplotlib.pyplot as plt

In [ ]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/books/book-crossings.zip

!unzip book-crossings.zip

books_filename = 'BX-Books.csv'
ratings_filename = 'BX-Book-Ratings.csv'

--2021-09-21 14:57:30--  https://cdn.freecodecamp.org/project-data/books/book-crossings.zip
Resolving cdn.freecodecamp.org (cdn.freecodecamp.org)... 104.26.3.33, 172.67.70.149, 104.26.2.33, ...
Connecting to cdn.freecodecamp.org (cdn.freecodecamp.org)|104.26.3.33|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 26085508 (25M) [application/zip]
Saving to: ‘book-crossings.zip’

book-crossings.zip  100%[===================>]  24.88M  13.7MB/s    in 1.8s    

2021-09-21 14:57:32 (13.7 MB/s) - ‘book-crossings.zip’ saved [26085508/26085508]

Archive:  book-crossings.zip
  inflating: BX-Book-Ratings.csv     
  inflating: BX-Books.csv            
  inflating: BX-Users.csv            


In [ ]:
# import csv data into dataframes
df_books = pd.read_csv(
    books_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['isbn', 'title', 'author'],
    usecols=['isbn', 'title', 'author'],
    dtype={'isbn': 'str', 'title': 'str', 'author': 'str'})

df_ratings = pd.read_csv(
    ratings_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['user', 'isbn', 'rating'],
    usecols=['user', 'isbn', 'rating'],
    dtype={'user': 'int32', 'isbn': 'str', 'rating': 'float32'})

In [ ]:
book_count = {}
for index, rating in df_ratings.iterrows():
  book_count[rating['isbn']] = book_count.get(rating['isbn'], 0) + 1

In [ ]:
user_count = {}
for index, rating in df_ratings.iterrows():
  user_count[rating['user']] = user_count.get(rating['user'], 0) + 1

In [ ]:
print(len(book_count))
print(len(user_count))

340556
105283


In [ ]:
#books to discard
books_discarted = [k for k, v in book_count.items() if v < 100]
print(len(books_discarted))

339825


In [ ]:
books_to_keep = [k for k,v in book_count.items() if v >= 100]
print(len(books_to_keep))

731


In [ ]:
#users to discard
users_discarted = [k for k,v in user_count.items() if v < 200]
print(len(users_discarted))

104378


In [ ]:
users_to_keep = [k for k,v in user_count.items() if v >= 200]
print(len(users_to_keep))

905


In [ ]:
print(len(df_ratings.index))

1149780


In [ ]:
df_clear_books = df_ratings[~df_ratings.isbn.isin(books_discarted)]
print(len(df_clear_books.index))

138461


In [ ]:
df_clear = df_clear_books[~df_clear_books.user.isin(users_discarted)]
print(len(df_clear.index))

49781


In [ ]:
df_merged = pd.merge(df_clear, df_books, how="left")
print(df_merged.head())
print(len(df_merged.index))

     user  ...              author
0  277427  ...   James Finn Garner
1  277427  ...  Barbara Kingsolver
2  277427  ...        Ann Patchett
3  277427  ...     Janet Evanovich
4  277427  ...       Benjamin Hoff

[5 rows x 5 columns]
49781


In [ ]:
df_merged = df_merged.drop(['isbn','author'], axis=1)
print(df_merged.head())

     user  rating                                              title
0  277427    10.0  Politically Correct Bedtime Stories: Modern Ta...
1  277427     0.0                      The Poisonwood Bible: A Novel
2  277427     0.0                                 Bel Canto: A Novel
3  277427     9.0  One for the Money (Stephanie Plum Novels (Pape...
4  277427     0.0                                    The Tao of Pooh


In [ ]:
df_merged = df_merged.drop_duplicates(subset=["title","user"],keep="first");
print(df_merged.head())

     user  rating                                              title
0  277427    10.0  Politically Correct Bedtime Stories: Modern Ta...
1  277427     0.0                      The Poisonwood Bible: A Novel
2  277427     0.0                                 Bel Canto: A Novel
3  277427     9.0  One for the Money (Stephanie Plum Novels (Pape...
4  277427     0.0                                    The Tao of Pooh


In [ ]:
df_pivot = df_merged.pivot(index='title',columns='user',values='rating').fillna(0)
df_matrix = csr_matrix(df_pivot.values)
print(df_pivot.head())

user                 254     2276    2766    ...  277478  277639  278418
title                                        ...                        
NaN                     8.0     0.0     0.0  ...     0.0     0.0     0.0
1984                    9.0     0.0     0.0  ...     0.0     0.0     0.0
1st to Die: A Novel     0.0     0.0     0.0  ...     0.0     0.0     0.0
2nd Chance              0.0    10.0     0.0  ...     0.0     0.0     0.0
4 Blondes               0.0     0.0     0.0  ...     0.0     0.0     0.0

[5 rows x 888 columns]


In [ ]:
knn = NearestNeighbors(metric='cosine').fit(df_matrix)

In [ ]:
# function to return recommended books - this will be tested
def get_recommends(book = ""):
  X = df_pivot[df_pivot.index == book]
  X = X.to_numpy().reshape(1,-1)
  distances, indices = knn.kneighbors(X, 5)
  recommended_books = []
  recommended_books.append(book)

  closest_books = []
  for i in range(4,0,-1):
    book = df_pivot.iloc[[indices.flatten()[i]]]
    closest_books.append([book.index.values[0], distances.flatten()[i]])

  recommended_books.append(closest_books)
  return recommended_books

In [ ]:
get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")

["Where the Heart Is (Oprah's Book Club (Paperback))",
 [['The Weight of Water', 0.77085835],
  ['The Surgeon', 0.7699411],
  ['I Know This Much Is True', 0.7677075],
  ['The Lovely Bones: A Novel', 0.7234864]]]

Use the cell below to test your function. The `test_book_recommendation()` function will inform you if you passed the challenge or need to keep trying.

In [ ]:
books = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
print(books)

def test_book_recommendation():
  test_pass = True
  recommends = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
  if recommends[0] != "Where the Heart Is (Oprah's Book Club (Paperback))":
    test_pass = False
  recommended_books = ["I'll Be Seeing You", 'The Weight of Water', 'The Surgeon', 'I Know This Much Is True']
  recommended_books_dist = [0.8, 0.77, 0.77, 0.77]
  for i in range(2): 
    if recommends[1][i][0] not in recommended_books:
      test_pass = False
    if abs(recommends[1][i][1] - recommended_books_dist[i]) >= 0.05:
      test_pass = False
  if test_pass:
    print("You passed the challenge! 🎉🎉🎉🎉🎉")
  else:
    print("You haven't passed yet. Keep trying!")

test_book_recommendation()

["Where the Heart Is (Oprah's Book Club (Paperback))", [['The Weight of Water', 0.77085835], ['The Surgeon', 0.7699411], ['I Know This Much Is True', 0.7677075], ['The Lovely Bones: A Novel', 0.7234864]]]
You passed the challenge! 🎉🎉🎉🎉🎉
